In [1]:
import cv2 as cv
import numpy as np
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit

In [2]:
b_gpu = gpuarray.to_gpu(np.random.randn(4, 4).astype(np.float32))

In [3]:
b_doubled = (2 * b_gpu).get()

In [4]:
print b_gpu
print b_doubled

[[-0.7436429  -1.8159572   0.28205386  1.7937574 ]
 [-0.4242038   0.8123264   0.8644576   0.00816789]
 [-0.44237193 -0.27193326 -1.2570636   0.20925684]
 [ 0.15309955  0.4520917  -0.55670154  1.7599062 ]]
[[-1.4872859  -3.6319144   0.5641077   3.5875149 ]
 [-0.8484076   1.6246527   1.7289152   0.01633577]
 [-0.88474387 -0.5438665  -2.5141273   0.4185137 ]
 [ 0.3061991   0.9041834  -1.1134031   3.5198123 ]]


In [5]:
PATH = "/home/nvidia-mm/Workspace/frame_diff_py/lena.jpg"
I = cv.imread(PATH,cv.IMREAD_COLOR)
print I.shape

(256, 256, 3)


In [6]:
# cv.imshow('image',I)
# if 27 == cv.waitKey(0):
#     cv.destroyAllWindows()

In [7]:
I_gpu_a = gpuarray.to_gpu(I.astype(np.float32))
I_gpu_b = gpuarray.to_gpu(I.astype(np.float32))
I_div = (I_gpu_a * 2 - I_gpu_b).get()
print I_gpu_a[0][0][0],'-',I_div[0][0][0]

120.0 - 120.0


In [8]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
import numpy as np

In [9]:
a = np.random.randn(4, 4)
a = a.astype(np.float32)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)

In [10]:
mod = SourceModule("""
        __global__ void doublify(float *a)
        {
          int idx = threadIdx.x + threadIdx.y*4;
          a[idx] *= 2;
        }
        __global__ void ImgSub(float **A, float **B, int size)
        {
          int idx = blockIdx.x*blockDim.x + threadIdx.x;
          int idy = blockIdx.y*blockDim.y + threadIdx.y;
          float tmp = 0;
          if(idx < size && idy < size){
            tmp = B[idx][idy];
            B[idx][idy] = A[idx][idy];
            if(A[idx][idy] > tmp){
              if(A[idx][idy] - tmp > 25){
                A[idx][idy] = 0;
              }
            }else{
              if(tmp - A[idx][idy] > 25){
                A[idx][idy] = 0;
              }
            }
          }
        }
        """)

In [11]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))
a_doubled = np.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)

print a_doubled
print a

[[-0.26412612 -2.599057   -1.8459568  -1.5173056 ]
 [-0.22525242 -1.240282   -0.01318387  1.2576464 ]
 [-0.43277848 -0.35822362 -2.4905665   0.18955706]
 [ 0.64920217 -0.37311763 -2.157971    4.828526  ]]
[[-0.13206306 -1.2995285  -0.9229784  -0.7586528 ]
 [-0.11262621 -0.620141   -0.00659193  0.6288232 ]
 [-0.21638924 -0.17911181 -1.2452832   0.09477853]
 [ 0.32460108 -0.18655881 -1.0789855   2.414263  ]]


In [12]:
img = cv.imread(PATH,cv.IMREAD_GRAYSCALE)
img = img.astype(np.float32)
# img_linear = np.ravel(img,'C')
img_res = np.zeros(img.shape).astype(np.float32)
w = np.int32(256)
h = np.int32(256)

img_a_gpu = cuda.mem_alloc(img.nbytes)
img_b_gpu = cuda.mem_alloc(img.nbytes)
cuda.memcpy_htod(img_a_gpu, img)
cuda.memcpy_htod(img_b_gpu, img)
func = mod.get_function("ImgSub")

In [13]:
func(img_a_gpu, img_b_gpu, w, block=(16,16,2), grid=(16,16))
print type(img_a_gpu)
# cuda.memcpy_dtoh(img_res, img_a_gpu)
# img_res = cuda.from_device(img_a_gpu,img.shape,np.uint8,order='C')
img_res = img_res.astype(np.uint8)
cv.imshow('img_moving', img_res)
if 27 == cv.waitKey(0):
    cv.destroyAllWindows()

<class 'pycuda._driver.DeviceAllocation'>


In [16]:
import pycuda.gpuarray as gpuarray
import pycuda.cumath as cmath
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import cv2 as cv

PATH = "/home/nvidia-mm/Workspace/frame_diff_py/lena.jpg"
# I = cv.imread(PATH,cv.IMREAD_COLOR)
# I_gpu_a = gpuarray.to_gpu(I.astype(np.float32))
# I_gpu_b = gpuarray.to_gpu(I.astype(np.float32))
# I_div = (I_gpu_a * 2 - I_gpu_b).get()
# print I_gpu_a[0][0][0],'-',I_div[0][0][0]

A = cv.imread(PATH,cv.IMREAD_COLOR)
img_ori_gpu = gpuarray.to_gpu(A.astype(np.float32))
img_res_gpu = gpuarray.to_gpu(A.astype(np.float32))
img_buf_gpu = gpuarray.empty_like(img_ori_gpu)
img_buf_gpu = cmath.fabs(img_ori_gpu - img_res_gpu)
MASK = img_buf_gpu.get()
print MASK.shape

LaunchError: cuMemAlloc failed: unspecified launch failure